In [1]:
import random
import os
import os.path as osp
import re
import pickle
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
import seaborn as sns
import pickle
import torch
import glob
from scipy.stats import entropy

pd.set_option('display.expand_frame_repr', False)  # DataFrameを改行せずに表示
pd.set_option('display.max_columns', None)  # すべての列を表示

In [2]:
def extract_finalQP(filename):
    match = re.search(r'2ndQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None


def extract_1stQP(filename):
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None


def ratio_double_compressed(mean_difference, final_QP):
    # mean_difference = mean_difference[0]
    # final_QP = final_QP[0]
    clamped_mean_difference = np.maximum(mean_difference, -0.01)
    
    #全体のエネルギーを計算
    energy = np.sum(np.square(clamped_mean_difference))
    # energy = np.sum(np.square(mean_difference))
    
    #QP2より右側のエネルギーを計算
    right_energy = np.sum(np.square(clamped_mean_difference[final_QP+1:52]))

        
    # エネルギー比を計算して閾値と比較
    if energy > 0:
        return right_energy / energy
    
    else:
        return 0

    
def is_double_compressed(mean_difference, final_QP, threshold):
    mean_difference = mean_difference[0]
    final_QP = final_QP[0]
    clamped_mean_difference = np.maximum(mean_difference, -0.01)
    
    #全体のエネルギーを計算
    energy = np.sum(np.square(clamped_mean_difference))
    # energy = np.sum(np.square(mean_difference))
    
    #QP2より右側のエネルギーを計算
    right_energy = np.sum(np.square(clamped_mean_difference[final_QP+1:52]))
    # right_energy = np.sum(np.square(mean_difference[final_QP+1:52]))
    
    # print('energy: ', energy)
    # print('R-energy: ', right_energy)
    # print('Ratio: ', right_energy / energy)
    
    
    # エネルギー比を計算して閾値と比較
    if energy <= 0:
        return -1
    
    elif (right_energy / energy) != 0 and (right_energy / energy) > threshold:
        return True
    
    elif (right_energy / energy) != 0 and (right_energy / energy) <= threshold:
        return False
    
    else:
        return -1

def calculate_mae(file_path):
    try:
        with open(file_path, 'rb') as file:
            loaded_data, loaded_data_shifted = pickle.load(file)
    except Exception as e:
        print(f"Error occurred while loading {file_path}: {e}")
        return None

    # タプル内のリストを抽出
    original_mae = np.array(loaded_data)
    shifted_mae = np.array(loaded_data_shifted)

    # Coding ghostを計算してリストに格納する
    mae_difference = shifted_mae - original_mae
    
    # mae_differenceの各要素においてマイナスの値を0に変換
    # mae_difference_positive = np.maximum(mae_difference, 0)
    
    return mae_difference

In [3]:
rootpath_csv = "/Prove/Yoshihisa/HEIF_ghost/HEIF_IMAGES_CSV/"

train_list1 = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"]

train_list2 = ["31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48", "49", "50", "51", "52", "53", "54", "55", "56", "57", "58", "59", "60"]

train_list3 = ["61", "62", "63", "64", "65", "66", "67", "68", "69", "70", "71", "72", "73", "74", "75", "76", "77", "78", "79", "80", "81", "82", "83", "84", "85", "86", "87", "88", "89", "90"]

train_list4 = ["91", "92", "93", "94", "95", "96", "97", "98", "99", "100", "101", "102", "103", "104", "105", "106", "107", "108", "109", "110", "111", "112", "113", "114", "115", "116", "117", "118", "119", "120"]

train_list5 = ["121", "122", "123", "124", "125", "126", "127", "128", "129", "130", "131", "132", "133", "134", "135", "136", "137", "138", "139", "140", "141", "142", "143", "144", "145", "146", "147", "148", "149", "150"]

train_list6 = ["151", "152", "153", "154", "155", "156", "157", "158", "159", "160", "161", "162", "163", "164", "165", "166", "167", "168", "169", "170", "171", "172", "173", "174", "175", "176", "177", "178", "179", "180"]

train_list7 = ["181", "182", "183", "184", "185", "186", "187", "188", "189", "190", "191", "192", "193", "194", "195", "196", "197", "198", "199", "200", "201", "202", "203", "204", "205", "206", "207", "208", "209", "210"]

train_list8 = ["211", "212", "213", "214", "215", "216", "217", "218", "219", "220", "221", "222", "223", "224", "225", "226", "227", "228", "229", "230", "231", "232", "233", "234", "235", "236", "237", "238", "239", "240"]

train_list9 = ["241", "242", "243", "244", "245", "246", "247", "248", "249", "250", "251", "252", "253", "254", "255", "256", "257", "258", "259", "260", "261", "262", "263", "264", "265", "266", "267", "268", "269", "270"]

train_list10 = ["271", "272", "273", "274", "275", "276", "277", "278", "279", "280", "281", "282", "283", "284", "285", "286", "287", "288", "289", "290", "291", "292", "293", "294", "295", "296", "297", "298", "299", "300"]


all_train_lists = [train_list1, train_list2, train_list3, train_list4, train_list5,
                   train_list6, train_list7, train_list8, train_list9, train_list10]

# すべてのリストを1つのリストに結合する
combined_train_list = sum(all_train_lists, [])

# リストの順序をランダムにシャッフルする
random.shuffle(combined_train_list)

# シャッフルされたリストを10個のグループに分割する
train_lists = [combined_train_list[i:i+30] for i in range(0, len(combined_train_list), 30)]
print(train_lists)

single_list1 = []
single_list2 = []
single_list3 = []
single_list4 = []
single_list5 = []
single_list6 = []
single_list7 = []
single_list8 = []
single_list9 = []
single_list10 = []

single_recompress_list1 = []
single_recompress_list2 = []
single_recompress_list3 = []
single_recompress_list4 = []
single_recompress_list5 = []
single_recompress_list6 = []
single_recompress_list7 = []
single_recompress_list8 = []
single_recompress_list9 = []
single_recompress_list10 = []

second_largeQP1_list1 = []
second_largeQP1_list2 = []
second_largeQP1_list3 = []
second_largeQP1_list4 = []
second_largeQP1_list5 = []
second_largeQP1_list6 = []
second_largeQP1_list7 = []
second_largeQP1_list8 = []
second_largeQP1_list9 = []
second_largeQP1_list10 = []

second_recompress_largeQP1_list1 = []
second_recompress_largeQP1_list2 = []
second_recompress_largeQP1_list3 = []
second_recompress_largeQP1_list4 = []
second_recompress_largeQP1_list5 = []
second_recompress_largeQP1_list6 = []
second_recompress_largeQP1_list7 = []
second_recompress_largeQP1_list8 = []
second_recompress_largeQP1_list9 = []
second_recompress_largeQP1_list10 = []

second_sameQP_list1 = []
second_sameQP_list2 = []
second_sameQP_list3 = []
second_sameQP_list4 = []
second_sameQP_list5 = []
second_sameQP_list6 = []
second_sameQP_list7 = []
second_sameQP_list8 = []
second_sameQP_list9 = []
second_sameQP_list10 = []

second_recompress_sameQP_list1 = []
second_recompress_sameQP_list2 = []
second_recompress_sameQP_list3 = []
second_recompress_sameQP_list4 = []
second_recompress_sameQP_list5 = []
second_recompress_sameQP_list6 = []
second_recompress_sameQP_list7 = []
second_recompress_sameQP_list8 = []
second_recompress_sameQP_list9 = []
second_recompress_sameQP_list10 = []

second_largeQP2_list1 = []
second_largeQP2_list2 = []
second_largeQP2_list3 = []
second_largeQP2_list4 = []
second_largeQP2_list5 = []
second_largeQP2_list6 = []
second_largeQP2_list7 = []
second_largeQP2_list8 = []
second_largeQP2_list9 = []
second_largeQP2_list10 = []

second_recompress_largeQP2_list1 = []
second_recompress_largeQP2_list2 = []
second_recompress_largeQP2_list3 = []
second_recompress_largeQP2_list4 = []
second_recompress_largeQP2_list5 = []
second_recompress_largeQP2_list6 = []
second_recompress_largeQP2_list7 = []
second_recompress_largeQP2_list8 = []
second_recompress_largeQP2_list9 = []
second_recompress_largeQP2_list10 = []


[['191', '10', '55', '118', '235', '241', '244', '200', '293', '273', '193', '71', '278', '212', '143', '79', '84', '275', '245', '46', '78', '268', '83', '276', '227', '230', '181', '122', '44', '188'], ['234', '163', '190', '152', '283', '90', '186', '286', '4', '231', '209', '109', '120', '194', '259', '51', '61', '136', '99', '290', '112', '104', '172', '158', '300', '284', '20', '165', '210', '72'], ['36', '68', '86', '221', '236', '295', '239', '94', '133', '149', '106', '285', '73', '141', '3', '96', '258', '57', '93', '13', '243', '110', '50', '170', '35', '113', '211', '82', '288', '129'], ['18', '228', '144', '117', '242', '107', '162', '132', '11', '176', '229', '166', '157', '98', '67', '294', '167', '29', '108', '253', '292', '33', '105', '277', '260', '192', '2', '184', '220', '246'], ['37', '75', '142', '48', '272', '19', '178', '289', '248', '199', '185', '77', '238', '64', '76', '8', '135', '124', '15', '134', '125', '223', '52', '269', '92', '196', '251', '203', '41',

In [4]:
def process_train_lists(train_list, single_list, single_recompress_list, 
                        second_largeQP1_list, second_recompress_largeQP1_list, 
                        second_sameQP_list, second_recompress_sameQP_list,
                        second_largeQP2_list, second_recompress_largeQP2_list):
    
    for image in train_list:
        single_path = osp.join(rootpath_csv+f'HEIF_images_single_csv/{image}_*')
        single_recompress_path = osp.join(rootpath_csv+f'HEIF_images_second_sameQP_csv/{image}_*')
        
        second_largeQP1_path = osp.join(rootpath_csv+f'HEIF_images_second_csv/{image}_*')
        second_recompress_largeQP1_path = osp.join(rootpath_csv+f'HEIF_images_triple_csv/{image}_*')
        
        second_sameQP_path = osp.join(rootpath_csv+f'HEIF_images_second_sameQP_csv/{image}_*')
        second_recompress_sameQP_path = osp.join(rootpath_csv+f'HEIF_images_triple_sameQP_csv/{image}_*')
        
        second_largeQP2_path = osp.join(rootpath_csv+f'HEIF_images_second_largeQP_csv/{image}_*')
        second_recompress_largeQP2_path = osp.join(rootpath_csv+f'HEIF_images_triple_largeQP_csv/{image}_*')
        

        for path in sorted(glob.glob(single_path)):
            single_list.append(path)
        for path in sorted(glob.glob(single_recompress_path)):
            single_recompress_list.append(path)
            
        for path in sorted(glob.glob(second_largeQP1_path)):
            second_largeQP1_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP1_path)):
            second_recompress_largeQP1_list.append(path)
                
        for path in sorted(glob.glob(second_sameQP_path)):
            second_sameQP_list.append(path)
        for path in sorted(glob.glob(second_recompress_sameQP_path)):
            second_recompress_sameQP_list.append(path)
            
        for path in sorted(glob.glob(second_largeQP2_path)):
            second_largeQP2_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP2_path)):
            second_recompress_largeQP2_list.append(path)
            

# train_lists = [train_list1, train_list2, train_list3, train_list4, train_list5, train_list6, train_list7, train_list8, train_list9, train_list10]

single_lists = [single_list1, single_list2,
                single_list3, single_list4,
                single_list5, single_list6,
                single_list7, single_list8,
                single_list9, single_list10]

single_recompress_lists = [single_recompress_list1, single_recompress_list2,
                           single_recompress_list3, single_recompress_list4,
                           single_recompress_list5, single_recompress_list6,
                           single_recompress_list7, single_recompress_list8,
                           single_recompress_list9, single_recompress_list10]


second_largeQP1_lists = [second_largeQP1_list1, second_largeQP1_list2,
                        second_largeQP1_list3, second_largeQP1_list4,
                        second_largeQP1_list5, second_largeQP1_list6,
                        second_largeQP1_list7, second_largeQP1_list8,
                        second_largeQP1_list9, second_largeQP1_list10]

second_recompress_largeQP1_lists = [second_recompress_largeQP1_list1, second_recompress_largeQP1_list2,
                           second_recompress_largeQP1_list3, second_recompress_largeQP1_list4,
                           second_recompress_largeQP1_list5, second_recompress_largeQP1_list6,
                           second_recompress_largeQP1_list7, second_recompress_largeQP1_list8,
                           second_recompress_largeQP1_list9, second_recompress_largeQP1_list10]


second_sameQP_lists = [second_sameQP_list1, second_sameQP_list2,
                        second_sameQP_list3, second_sameQP_list4,
                        second_sameQP_list5, second_sameQP_list6,
                        second_sameQP_list7, second_sameQP_list8,
                        second_sameQP_list9, second_sameQP_list10]

second_recompress_sameQP_lists = [second_recompress_sameQP_list1, second_recompress_sameQP_list2,
                           second_recompress_sameQP_list3, second_recompress_sameQP_list4,
                           second_recompress_sameQP_list5, second_recompress_sameQP_list6,
                           second_recompress_sameQP_list7, second_recompress_sameQP_list8,
                           second_recompress_sameQP_list9, second_recompress_sameQP_list10]


second_largeQP2_lists = [second_largeQP2_list1, second_largeQP2_list2,
                        second_largeQP2_list3, second_largeQP2_list4,
                        second_largeQP2_list5, second_largeQP2_list6,
                        second_largeQP2_list7, second_largeQP2_list8,
                        second_largeQP2_list9, second_largeQP2_list10]

second_recompress_largeQP2_lists = [second_recompress_largeQP2_list1, second_recompress_largeQP2_list2,
                           second_recompress_largeQP2_list3, second_recompress_largeQP2_list4,
                           second_recompress_largeQP2_list5, second_recompress_largeQP2_list6,
                           second_recompress_largeQP2_list7, second_recompress_largeQP2_list8,
                           second_recompress_largeQP2_list9, second_recompress_largeQP2_list10]



for train_list, single_list, single_recompress_list, second_largeQP1_list, second_recompress_largeQP1_list, second_sameQP_list, second_recompress_sameQP_list, second_largeQP2_list, second_recompress_largeQP2_list in zip(train_lists, 
                                                                                                                                                                                                                           single_lists,
                                                                                                                                                                                                                           single_recompress_lists,
                                                                                                                                                                                                                           second_largeQP1_lists,
                                                                                                                                                                                                                           second_recompress_largeQP1_lists,
                                                                                                                                                                                                                           second_sameQP_lists,
                                                                                                                                                                                                                           second_recompress_sameQP_lists,
                                                                                                                                                                                                                           second_largeQP2_lists,
                                                                                                                                                                                                                           second_recompress_largeQP2_lists
                                                                                                                                                                                                                          ):
    process_train_lists(train_list, single_list, single_recompress_list, 
                        second_largeQP1_list, second_recompress_largeQP1_list, 
                        second_sameQP_list, second_recompress_sameQP_list,
                        second_largeQP2_list, second_recompress_largeQP2_list)


In [5]:
print(len(single_lists[6]))
print(len(single_recompress_lists[6]))
print(len(second_largeQP1_lists[6]))
print(len(second_recompress_largeQP1_lists[6]))
print(len(second_sameQP_lists[6]))
print(len(second_recompress_sameQP_lists[6]))
print(len(second_largeQP2_lists[6]))
print(len(second_recompress_largeQP2_lists[6]))

300
300
1710
1710
300
300
1170
1170


In [6]:
single_list1 = []
single_list2 = []
single_list3 = []
single_list4 = []
single_list5 = []
single_list6 = []
single_list7 = []
single_list8 = []
single_list9 = []
single_list10 = []

single_recompress_list1 = []
single_recompress_list2 = []
single_recompress_list3 = []
single_recompress_list4 = []
single_recompress_list5 = []
single_recompress_list6 = []
single_recompress_list7 = []
single_recompress_list8 = []
single_recompress_list9 = []
single_recompress_list10 = []

second_largeQP1_list1 = []
second_largeQP1_list2 = []
second_largeQP1_list3 = []
second_largeQP1_list4 = []
second_largeQP1_list5 = []
second_largeQP1_list6 = []
second_largeQP1_list7 = []
second_largeQP1_list8 = []
second_largeQP1_list9 = []
second_largeQP1_list10 = []

second_recompress_largeQP1_list1 = []
second_recompress_largeQP1_list2 = []
second_recompress_largeQP1_list3 = []
second_recompress_largeQP1_list4 = []
second_recompress_largeQP1_list5 = []
second_recompress_largeQP1_list6 = []
second_recompress_largeQP1_list7 = []
second_recompress_largeQP1_list8 = []
second_recompress_largeQP1_list9 = []
second_recompress_largeQP1_list10 = []

second_sameQP_list1 = []
second_sameQP_list2 = []
second_sameQP_list3 = []
second_sameQP_list4 = []
second_sameQP_list5 = []
second_sameQP_list6 = []
second_sameQP_list7 = []
second_sameQP_list8 = []
second_sameQP_list9 = []
second_sameQP_list10 = []

second_recompress_sameQP_list1 = []
second_recompress_sameQP_list2 = []
second_recompress_sameQP_list3 = []
second_recompress_sameQP_list4 = []
second_recompress_sameQP_list5 = []
second_recompress_sameQP_list6 = []
second_recompress_sameQP_list7 = []
second_recompress_sameQP_list8 = []
second_recompress_sameQP_list9 = []
second_recompress_sameQP_list10 = []

second_largeQP2_list1 = []
second_largeQP2_list2 = []
second_largeQP2_list3 = []
second_largeQP2_list4 = []
second_largeQP2_list5 = []
second_largeQP2_list6 = []
second_largeQP2_list7 = []
second_largeQP2_list8 = []
second_largeQP2_list9 = []
second_largeQP2_list10 = []

second_recompress_largeQP2_list1 = []
second_recompress_largeQP2_list2 = []
second_recompress_largeQP2_list3 = []
second_recompress_largeQP2_list4 = []
second_recompress_largeQP2_list5 = []
second_recompress_largeQP2_list6 = []
second_recompress_largeQP2_list7 = []
second_recompress_largeQP2_list8 = []
second_recompress_largeQP2_list9 = []
second_recompress_largeQP2_list10 = []

In [7]:
rootpath_pkl = "/Prove/Yoshihisa/HEIF_ghost/PKL/"


def process_train_lists2(train_list, single_list, single_recompress_list, 
                        second_largeQP1_list, second_recompress_largeQP1_list, 
                        second_sameQP_list, second_recompress_sameQP_list,
                        second_largeQP2_list, second_recompress_largeQP2_list):
    
    for image in train_list:
        single_path = osp.join(rootpath_pkl+f'pkl_single/{image}_*')
        single_recompress_path = osp.join(rootpath_pkl+f'pkl_second_sameQP/{image}_*')
        
        second_largeQP1_path = osp.join(rootpath_pkl+f'pkl_second/{image}_*')
        second_recompress_largeQP1_path = osp.join(rootpath_pkl+f'pkl_triple/{image}_*')
        
        second_sameQP_path = osp.join(rootpath_pkl+f'pkl_second_sameQP/{image}_*')
        second_recompress_sameQP_path = osp.join(rootpath_pkl+f'pkl_triple_sameQP/{image}_*')
        
        second_largeQP2_path = osp.join(rootpath_pkl+f'pkl_second_largeQP/{image}_*')
        second_recompress_largeQP2_path = osp.join(rootpath_pkl+f'pkl_triple_largeQP/{image}_*')
        

        for path in sorted(glob.glob(single_path)):
            single_list.append(path)
        for path in sorted(glob.glob(single_recompress_path)):
            single_recompress_list.append(path)
            
        for path in sorted(glob.glob(second_largeQP1_path)):
            second_largeQP1_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP1_path)):
            second_recompress_largeQP1_list.append(path)
                
        for path in sorted(glob.glob(second_sameQP_path)):
            second_sameQP_list.append(path)
        for path in sorted(glob.glob(second_recompress_sameQP_path)):
            second_recompress_sameQP_list.append(path)
            
        for path in sorted(glob.glob(second_largeQP2_path)):
            second_largeQP2_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP2_path)):
            second_recompress_largeQP2_list.append(path)
            

# train_lists = [train_list1, train_list2, train_list3, train_list4, train_list5, train_list6, train_list7, train_list8, train_list9, train_list10]

single_listsA = [single_list1, single_list2,
                single_list3, single_list4,
                single_list5, single_list6,
                single_list7, single_list8,
                single_list9, single_list10]

single_recompress_listsA = [single_recompress_list1, single_recompress_list2,
                           single_recompress_list3, single_recompress_list4,
                           single_recompress_list5, single_recompress_list6,
                           single_recompress_list7, single_recompress_list8,
                           single_recompress_list9, single_recompress_list10]


second_largeQP1_listsA = [second_largeQP1_list1, second_largeQP1_list2,
                        second_largeQP1_list3, second_largeQP1_list4,
                        second_largeQP1_list5, second_largeQP1_list6,
                        second_largeQP1_list7, second_largeQP1_list8,
                        second_largeQP1_list9, second_largeQP1_list10]

second_recompress_largeQP1_listsA = [second_recompress_largeQP1_list1, second_recompress_largeQP1_list2,
                           second_recompress_largeQP1_list3, second_recompress_largeQP1_list4,
                           second_recompress_largeQP1_list5, second_recompress_largeQP1_list6,
                           second_recompress_largeQP1_list7, second_recompress_largeQP1_list8,
                           second_recompress_largeQP1_list9, second_recompress_largeQP1_list10]


second_sameQP_listsA = [second_sameQP_list1, second_sameQP_list2,
                        second_sameQP_list3, second_sameQP_list4,
                        second_sameQP_list5, second_sameQP_list6,
                        second_sameQP_list7, second_sameQP_list8,
                        second_sameQP_list9, second_sameQP_list10]

second_recompress_sameQP_listsA = [second_recompress_sameQP_list1, second_recompress_sameQP_list2,
                           second_recompress_sameQP_list3, second_recompress_sameQP_list4,
                           second_recompress_sameQP_list5, second_recompress_sameQP_list6,
                           second_recompress_sameQP_list7, second_recompress_sameQP_list8,
                           second_recompress_sameQP_list9, second_recompress_sameQP_list10]


second_largeQP2_listsA = [second_largeQP2_list1, second_largeQP2_list2,
                        second_largeQP2_list3, second_largeQP2_list4,
                        second_largeQP2_list5, second_largeQP2_list6,
                        second_largeQP2_list7, second_largeQP2_list8,
                        second_largeQP2_list9, second_largeQP2_list10]

second_recompress_largeQP2_listsA = [second_recompress_largeQP2_list1, second_recompress_largeQP2_list2,
                           second_recompress_largeQP2_list3, second_recompress_largeQP2_list4,
                           second_recompress_largeQP2_list5, second_recompress_largeQP2_list6,
                           second_recompress_largeQP2_list7, second_recompress_largeQP2_list8,
                           second_recompress_largeQP2_list9, second_recompress_largeQP2_list10]



for train_list, single_list, single_recompress_list, second_largeQP1_list, second_recompress_largeQP1_list, second_sameQP_list, second_recompress_sameQP_list, second_largeQP2_list, second_recompress_largeQP2_list in zip(train_lists, 
                                                                                                                                                                                                                           single_listsA,
                                                                                                                                                                                                                           single_recompress_listsA,
                                                                                                                                                                                                                           second_largeQP1_listsA,
                                                                                                                                                                                                                           second_recompress_largeQP1_listsA,
                                                                                                                                                                                                                           second_sameQP_listsA,
                                                                                                                                                                                                                           second_recompress_sameQP_listsA,
                                                                                                                                                                                                                           second_largeQP2_listsA,
                                                                                                                                                                                                                           second_recompress_largeQP2_listsA
                                                                                                                                                                                                                          ):
    process_train_lists2(train_list, single_list, single_recompress_list, 
                        second_largeQP1_list, second_recompress_largeQP1_list, 
                        second_sameQP_list, second_recompress_sameQP_list,
                        second_largeQP2_list, second_recompress_largeQP2_list)

In [8]:
print(len(single_listsA[6]))
print(len(single_recompress_listsA[6]))
print(len(second_largeQP1_listsA[6]))
print(len(second_recompress_largeQP1_listsA[6]))
print(len(second_sameQP_listsA[6]))
print(len(second_recompress_sameQP_listsA[6]))
print(len(second_largeQP2_listsA[6]))
print(len(second_recompress_largeQP2_listsA[6]))

300
300
1710
1710
300
300
1170
1170


In [9]:
single_csv1 = list(zip(single_lists[0], single_listsA[0], single_recompress_lists[0], single_recompress_listsA[0]))
single_csv2 = list(zip(single_lists[1], single_listsA[1], single_recompress_lists[1], single_recompress_listsA[1]))
single_csv3 = list(zip(single_lists[2], single_listsA[2], single_recompress_lists[2], single_recompress_listsA[2]))
single_csv4 = list(zip(single_lists[3], single_listsA[3], single_recompress_lists[3], single_recompress_listsA[3]))
single_csv5 = list(zip(single_lists[4], single_listsA[4], single_recompress_lists[4], single_recompress_listsA[4]))
single_csv6 = list(zip(single_lists[5], single_listsA[5], single_recompress_lists[5], single_recompress_listsA[5]))
single_csv7 = list(zip(single_lists[6], single_listsA[6], single_recompress_lists[6], single_recompress_listsA[6]))
single_csv8 = list(zip(single_lists[7], single_listsA[7], single_recompress_lists[7], single_recompress_listsA[7]))
single_csv9 = list(zip(single_lists[8], single_listsA[8], single_recompress_lists[8], single_recompress_listsA[8]))
single_csv10 = list(zip(single_lists[9], single_listsA[9], single_recompress_lists[9], single_recompress_listsA[9]))
print(len(single_csv7))

second_largeQP1_csv1 = list(zip(second_largeQP1_lists[0], second_largeQP1_listsA[0], second_recompress_largeQP1_lists[0], second_recompress_largeQP1_listsA[0]))
second_largeQP1_csv2 = list(zip(second_largeQP1_lists[1], second_largeQP1_listsA[1], second_recompress_largeQP1_lists[1], second_recompress_largeQP1_listsA[1]))
second_largeQP1_csv3 = list(zip(second_largeQP1_lists[2], second_largeQP1_listsA[2], second_recompress_largeQP1_lists[2], second_recompress_largeQP1_listsA[2]))
second_largeQP1_csv4 = list(zip(second_largeQP1_lists[3], second_largeQP1_listsA[3], second_recompress_largeQP1_lists[3], second_recompress_largeQP1_listsA[3]))
second_largeQP1_csv5 = list(zip(second_largeQP1_lists[4], second_largeQP1_listsA[4], second_recompress_largeQP1_lists[4], second_recompress_largeQP1_listsA[4]))
second_largeQP1_csv6 = list(zip(second_largeQP1_lists[5], second_largeQP1_listsA[5], second_recompress_largeQP1_lists[5], second_recompress_largeQP1_listsA[5]))
second_largeQP1_csv7 = list(zip(second_largeQP1_lists[6], second_largeQP1_listsA[6], second_recompress_largeQP1_lists[6], second_recompress_largeQP1_listsA[6]))
second_largeQP1_csv8 = list(zip(second_largeQP1_lists[7], second_largeQP1_listsA[7], second_recompress_largeQP1_lists[7], second_recompress_largeQP1_listsA[7]))
second_largeQP1_csv9 = list(zip(second_largeQP1_lists[8], second_largeQP1_listsA[8], second_recompress_largeQP1_lists[8], second_recompress_largeQP1_listsA[8]))
second_largeQP1_csv10 = list(zip(second_largeQP1_lists[9], second_largeQP1_listsA[9], second_recompress_largeQP1_lists[9], second_recompress_largeQP1_listsA[9]))

second_largeQP1_csv1 = random.sample(second_largeQP1_csv1, 100)
second_largeQP1_csv2 = random.sample(second_largeQP1_csv2, 100)
second_largeQP1_csv3 = random.sample(second_largeQP1_csv3, 100)
second_largeQP1_csv4 = random.sample(second_largeQP1_csv4, 100)
second_largeQP1_csv5 = random.sample(second_largeQP1_csv5, 100)
second_largeQP1_csv6 = random.sample(second_largeQP1_csv6, 100)
second_largeQP1_csv7 = random.sample(second_largeQP1_csv7, 100)
second_largeQP1_csv8 = random.sample(second_largeQP1_csv8, 100)
second_largeQP1_csv9 = random.sample(second_largeQP1_csv9, 100)
second_largeQP1_csv10 = random.sample(second_largeQP1_csv10, 100)
print(len(second_largeQP1_csv7))

second_sameQP_csv1 = list(zip(second_sameQP_lists[0], second_sameQP_listsA[0], second_recompress_sameQP_lists[0], second_recompress_sameQP_listsA[0]))
second_sameQP_csv2 = list(zip(second_sameQP_lists[1], second_sameQP_listsA[1], second_recompress_sameQP_lists[1], second_recompress_sameQP_listsA[1]))
second_sameQP_csv3 = list(zip(second_sameQP_lists[2], second_sameQP_listsA[2], second_recompress_sameQP_lists[2], second_recompress_sameQP_listsA[2]))
second_sameQP_csv4 = list(zip(second_sameQP_lists[3], second_sameQP_listsA[3], second_recompress_sameQP_lists[3], second_recompress_sameQP_listsA[3]))
second_sameQP_csv5 = list(zip(second_sameQP_lists[4], second_sameQP_listsA[4], second_recompress_sameQP_lists[4], second_recompress_sameQP_listsA[4]))
second_sameQP_csv6 = list(zip(second_sameQP_lists[5], second_sameQP_listsA[5], second_recompress_sameQP_lists[5], second_recompress_sameQP_listsA[5]))
second_sameQP_csv7 = list(zip(second_sameQP_lists[6], second_sameQP_listsA[6], second_recompress_sameQP_lists[6], second_recompress_sameQP_listsA[6]))
second_sameQP_csv8 = list(zip(second_sameQP_lists[7], second_sameQP_listsA[7], second_recompress_sameQP_lists[7], second_recompress_sameQP_listsA[7]))
second_sameQP_csv9 = list(zip(second_sameQP_lists[8], second_sameQP_listsA[8], second_recompress_sameQP_lists[8], second_recompress_sameQP_listsA[8]))
second_sameQP_csv10 = list(zip(second_sameQP_lists[9], second_sameQP_listsA[9], second_recompress_sameQP_lists[9], second_recompress_sameQP_listsA[9]))

second_sameQP_csv1 = random.sample(second_sameQP_csv1, 100)
second_sameQP_csv2 = random.sample(second_sameQP_csv2, 100)
second_sameQP_csv3 = random.sample(second_sameQP_csv3, 100)
second_sameQP_csv4 = random.sample(second_sameQP_csv4, 100)
second_sameQP_csv5 = random.sample(second_sameQP_csv5, 100)
second_sameQP_csv6 = random.sample(second_sameQP_csv6, 100)
second_sameQP_csv7 = random.sample(second_sameQP_csv7, 100)
second_sameQP_csv8 = random.sample(second_sameQP_csv8, 100)
second_sameQP_csv9 = random.sample(second_sameQP_csv9, 100)
second_sameQP_csv10 = random.sample(second_sameQP_csv10, 100)
print(len(second_sameQP_csv7))

second_largeQP2_csv1 = list(zip(second_largeQP2_lists[0], second_largeQP2_listsA[0], second_recompress_largeQP2_lists[0], second_recompress_largeQP2_listsA[0]))
second_largeQP2_csv2 = list(zip(second_largeQP2_lists[1], second_largeQP2_listsA[1], second_recompress_largeQP2_lists[1], second_recompress_largeQP2_listsA[1]))
second_largeQP2_csv3 = list(zip(second_largeQP2_lists[2], second_largeQP2_listsA[2], second_recompress_largeQP2_lists[2], second_recompress_largeQP2_listsA[2]))
second_largeQP2_csv4 = list(zip(second_largeQP2_lists[3], second_largeQP2_listsA[3], second_recompress_largeQP2_lists[3], second_recompress_largeQP2_listsA[3]))
second_largeQP2_csv5 = list(zip(second_largeQP2_lists[4], second_largeQP2_listsA[4], second_recompress_largeQP2_lists[4], second_recompress_largeQP2_listsA[4]))
second_largeQP2_csv6 = list(zip(second_largeQP2_lists[5], second_largeQP2_listsA[5], second_recompress_largeQP2_lists[5], second_recompress_largeQP2_listsA[5]))
second_largeQP2_csv7 = list(zip(second_largeQP2_lists[6], second_largeQP2_listsA[6], second_recompress_largeQP2_lists[6], second_recompress_largeQP2_listsA[6]))
second_largeQP2_csv8 = list(zip(second_largeQP2_lists[7], second_largeQP2_listsA[7], second_recompress_largeQP2_lists[7], second_recompress_largeQP2_listsA[7]))
second_largeQP2_csv9 = list(zip(second_largeQP2_lists[8], second_largeQP2_listsA[8], second_recompress_largeQP2_lists[8], second_recompress_largeQP2_listsA[8]))
second_largeQP2_csv10 = list(zip(second_largeQP2_lists[9], second_largeQP2_listsA[9], second_recompress_largeQP2_lists[9], second_recompress_largeQP2_listsA[9]))

second_largeQP2_csv1 = random.sample(second_largeQP2_csv1, 100)
second_largeQP2_csv2 = random.sample(second_largeQP2_csv2, 100)
second_largeQP2_csv3 = random.sample(second_largeQP2_csv3, 100)
second_largeQP2_csv4 = random.sample(second_largeQP2_csv4, 100)
second_largeQP2_csv5 = random.sample(second_largeQP2_csv5, 100)
second_largeQP2_csv6 = random.sample(second_largeQP2_csv6, 100)
second_largeQP2_csv7 = random.sample(second_largeQP2_csv7, 100)
second_largeQP2_csv8 = random.sample(second_largeQP2_csv8, 100)
second_largeQP2_csv9 = random.sample(second_largeQP2_csv9, 100)
second_largeQP2_csv10 = random.sample(second_largeQP2_csv10, 100)
print(len(second_largeQP2_csv7))


train_csv_list1 = single_csv1 + second_largeQP1_csv1 + second_sameQP_csv1 + second_largeQP2_csv1
train_csv_list2 = single_csv2 + second_largeQP1_csv2 + second_sameQP_csv2 + second_largeQP2_csv2
train_csv_list3 = single_csv3 + second_largeQP1_csv3 + second_sameQP_csv3 + second_largeQP2_csv3
train_csv_list4 = single_csv4 + second_largeQP1_csv4 + second_sameQP_csv4 + second_largeQP2_csv4
train_csv_list5 = single_csv5 + second_largeQP1_csv5 + second_sameQP_csv5 + second_largeQP2_csv5
train_csv_list6 = single_csv6 + second_largeQP1_csv6 + second_sameQP_csv6 + second_largeQP2_csv6
train_csv_list7 = single_csv7 + second_largeQP1_csv7 + second_sameQP_csv7 + second_largeQP2_csv7
train_csv_list8 = single_csv8 + second_largeQP1_csv8 + second_sameQP_csv8 + second_largeQP2_csv8
train_csv_list9 = single_csv9 + second_largeQP1_csv9 + second_sameQP_csv9 + second_largeQP2_csv9
train_csv_list10 = single_csv10 + second_largeQP1_csv10 + second_sameQP_csv10 + second_largeQP2_csv10

print("train_csv_list: ", len(train_csv_list7))

300
100
100
100
train_csv_list:  600


In [10]:
def laplace_smoothing(probabilities, alpha=1):
    """
    ラプラス平滑化を行う関数
    
    Args:
    probabilities (list): 平滑化する確率分布のリスト
    alpha (float): 平滑化パラメータ
    
    Returns:
    smoothed_probabilities (list): 平滑化された確率分布のリスト
    """
    total_count = sum(probabilities)
    num_elements = len(probabilities)
    
    smoothed_probabilities = [(count + alpha) / (total_count + alpha * num_elements) for count in probabilities]
    
    return smoothed_probabilities




def process_train_csv_lists(train_csv_list):
    pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
                  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

#     luminance_columns = ["LU1_0","LU1_1","LU1_2","LU1_3",
#                          "LU1_4","LU1_5","LU1_6","LU1_7",
#                          "LU1_8","LU1_9","LU1_10","LU1_11",
#                          "LU1_12","LU1_13","LU1_14","LU1_15",
#                          "LU1_16","LU1_17","LU1_18","LU1_19",
#                          "LU1_20","LU1_21","LU1_22","LU1_23",
#                          "LU1_24","LU1_25","LU1_26","LU1_27",
#                          "LU1_28","LU1_29","LU1_30","LU1_31",
#                          "LU1_32","LU1_33","LU1_34",
                         
#                          "LU2_0","LU2_1","LU2_2","LU2_3",
#                          "LU2_4","LU2_5","LU2_6","LU2_7",
#                          "LU2_8","LU2_9","LU2_10","LU2_11",
#                          "LU2_12","LU2_13","LU2_14","LU2_15",
#                          "LU2_16","LU2_17","LU2_18","LU2_19",
#                          "LU2_20","LU2_21","LU2_22","LU2_23",
#                          "LU2_24","LU2_25","LU2_26","LU2_27",
#                          "LU2_28","LU2_29","LU2_30","LU2_31",
#                          "LU2_32","LU2_33","LU2_34"]
    
    luminance_columns = ["LU1_0","LU1_1","LU1_9","LU1_10","LU1_11","LU1_25","LU1_26","LU1_27",
                         "LU2_0","LU2_1","LU2_9","LU2_10","LU2_11", "LU2_25","LU2_26","LU2_27"]

    chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                           "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]
    
    
    
    label_columns = ["LABEL"]
    mae1_columns = [f"MAE1_{i}" for i in range(52)]
    mae2_columns = [f"MAE2_{i}" for i in range(52)]
    mae_columns = ["MAE"]
    final_qp_columns = ["FINAL_QP"]
    kl_divergence1 = ["KLD_PU"]
    kl_divergence2 = ["KLD_LUMA"]
    kl_divergence3 = ["KLD_CHROMA"]
    ratio_columns1 = ["RATIO1"]
    ratio_columns2 = ["RATIO2"]
    
    train_df1_1 = pd.DataFrame(columns=pu_columns)
    train_df1_2 = pd.DataFrame(columns=luminance_columns)
    train_df1_3 = pd.DataFrame(columns=chrominance_columns)
    LABEL = pd.DataFrame(columns=label_columns)
    RATIO1 = pd.DataFrame(columns=ratio_columns1)
    RATIO2 = pd.DataFrame(columns=ratio_columns2)
    train_df3 = pd.DataFrame(columns=mae1_columns)
    train_df4 = pd.DataFrame(columns=mae2_columns)
    MAE = pd.DataFrame(columns=mae_columns)
    FINAL_QP = pd.DataFrame(columns=final_qp_columns)
    kl_divergence_df1 = pd.DataFrame(columns=kl_divergence1)
    kl_divergence_df2 = pd.DataFrame(columns=kl_divergence2)
    kl_divergence_df3 = pd.DataFrame(columns=kl_divergence3)

    for path1, path2, path3, path4 in train_csv_list:
        label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
        train_pkl_list = [path2, path4]
        df1 = pd.read_csv(path1)
        df2 = pd.read_csv(path3)
        
        # 平滑化を行う
        probabilities_df1 = laplace_smoothing([df1.loc[i, "pu_counts"] for i in [0,1,2,3,4]])
        probabilities_df2 = laplace_smoothing([df2.loc[i, "pu_counts"] for i in [0,1,2,3,4]])
        kl_divergence1 = entropy(probabilities_df1, probabilities_df2)
        
        probabilities_df3 = laplace_smoothing([df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]])
        probabilities_df4 = laplace_smoothing([df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]])
        kl_divergence2 = entropy(probabilities_df3, probabilities_df4)
        
        probabilities_df5 = laplace_smoothing([df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]])
        probabilities_df6 = laplace_smoothing([df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]])
        kl_divergence3 = entropy(probabilities_df5, probabilities_df6)
        
        
        pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
        # lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
        lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
        ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]
        
        train_df1_1 = pd.concat([train_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
        train_df1_2= pd.concat([train_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
        train_df1_3 = pd.concat([train_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
        kl_divergence_df1 = pd.concat([kl_divergence_df1, pd.DataFrame({"KLD_PU": [kl_divergence1]})], ignore_index=True)
        kl_divergence_df2 = pd.concat([kl_divergence_df2, pd.DataFrame({"KLD_LUMA": [kl_divergence2]})], ignore_index=True)
        kl_divergence_df3 = pd.concat([kl_divergence_df3, pd.DataFrame({"KLD_CHROMA": [kl_divergence3]})], ignore_index=True)


        LABEL = pd.concat([LABEL, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

        final_QP = extract_finalQP(train_pkl_list[0])

        mae_d1 = calculate_mae(train_pkl_list[0])
        mae_d2 = calculate_mae(train_pkl_list[1])
        ratio1 = ratio_double_compressed(mae_d1, final_QP)
        ratio2 = ratio_double_compressed(mae_d2, final_QP)

        RATIO1 = pd.concat([RATIO1, pd.DataFrame({"RATIO1": [ratio1]})], ignore_index=True)
        RATIO2 = pd.concat([RATIO2, pd.DataFrame({"RATIO2": [ratio2]})], ignore_index=True)

        train_df3 = pd.concat([train_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)
        train_df4 = pd.concat([train_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
        MAE = pd.concat([MAE, pd.DataFrame({"MAE": [mae_d1]})], ignore_index=True)
        FINAL_QP = pd.concat([FINAL_QP, pd.DataFrame({"FINAL_QP": [final_QP]})], ignore_index=True)

    train_df1_1.reset_index(drop=True, inplace=True)
    train_df1_2.reset_index(drop=True, inplace=True)
    train_df1_3.reset_index(drop=True, inplace=True)
    LABEL.reset_index(drop=True, inplace=True)
    RATIO1.reset_index(drop=True, inplace=True)
    RATIO2.reset_index(drop=True, inplace=True)
    kl_divergence_df1.reset_index(drop=True, inplace=True)
    kl_divergence_df2.reset_index(drop=True, inplace=True)
    kl_divergence_df3.reset_index(drop=True, inplace=True)

    # train_df = pd.concat([train_df1_1, train_df1_2, train_df1_3, train_df3, train_df4], axis=1)
    train_df = pd.concat([FINAL_QP, kl_divergence_df1, kl_divergence_df2, kl_divergence_df3, RATIO1, RATIO2], axis=1)
    train_df_onlyGhost = pd.concat([FINAL_QP, train_df1_1, train_df1_2, train_df1_3, kl_divergence_df1, kl_divergence_df2, kl_divergence_df3, RATIO1, RATIO2], axis=1)

    return train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP


In [11]:
train_df1, train_df_onlyGhost1, LABEL1, MAE1, FINAL_QP1 = process_train_csv_lists(train_csv_list1)
train_df2, train_df_onlyGhost2, LABEL2, MAE2, FINAL_QP2 = process_train_csv_lists(train_csv_list2)
train_df3, train_df_onlyGhost3, LABEL3, MAE3, FINAL_QP3 = process_train_csv_lists(train_csv_list3)
train_df4, train_df_onlyGhost4, LABEL4, MAE4, FINAL_QP4 = process_train_csv_lists(train_csv_list4)
train_df5, train_df_onlyGhost5, LABEL5, MAE5, FINAL_QP5 = process_train_csv_lists(train_csv_list5)
train_df6, train_df_onlyGhost6, LABEL6, MAE6, FINAL_QP6 = process_train_csv_lists(train_csv_list6)
train_df7, train_df_onlyGhost7, LABEL7, MAE7, FINAL_QP7 = process_train_csv_lists(train_csv_list7)
train_df8, train_df_onlyGhost8, LABEL8, MAE8, FINAL_QP8 = process_train_csv_lists(train_csv_list8)
train_df9, train_df_onlyGhost9, LABEL9, MAE9, FINAL_QP9 = process_train_csv_lists(train_csv_list9)
train_df10, train_df_onlyGhost10, LABEL10, MAE10, FINAL_QP10 = process_train_csv_lists(train_csv_list10)

In [12]:
print(train_df1)

    FINAL_QP    KLD_PU  KLD_LUMA KLD_CHROMA    RATIO1    RATIO2
0         10  0.000457  0.000857   0.001671  0.117086  0.073309
1         16   0.00221  0.000518   0.001766  0.040061  0.022211
2         20  0.001292  0.000632   0.000502  0.027687  0.019464
3         24  0.000438  0.001074     0.0006  0.015096  0.008605
4         27    0.0001  0.000633   0.001434  0.238281  0.190728
..       ...       ...       ...        ...       ...       ...
595       32  0.000009  0.000547   0.012768  0.221121   0.21157
596       20  0.000245  0.020565   0.002304  0.310407  0.317506
597       32  0.000049  0.001894   0.000899   0.21278  0.202102
598       16  0.000355  0.000734   0.000192  0.016854  0.011868
599       24  0.000034  0.006753   0.000265  0.347295  0.348914

[600 rows x 6 columns]


In [13]:
# 各データフレームを結合
combined_train_df = pd.concat([train_df1, train_df2, train_df3, train_df4, train_df5, train_df6, train_df7, train_df8, train_df9, train_df10], ignore_index=True)
combined_train_df_onlyGhost = pd.concat([train_df_onlyGhost1, train_df_onlyGhost2, train_df_onlyGhost3, train_df_onlyGhost4, train_df_onlyGhost5, train_df_onlyGhost6, train_df_onlyGhost7, train_df_onlyGhost8, train_df_onlyGhost9, train_df_onlyGhost10], ignore_index=True)
combined_LABEL = pd.concat([LABEL1, LABEL2, LABEL3, LABEL4, LABEL5, LABEL6, LABEL7, LABEL8, LABEL9, LABEL10], ignore_index=True)
combined_MAE = pd.concat([MAE1, MAE2, MAE3, MAE4, MAE5, MAE6, MAE7, MAE8, MAE9, MAE10], ignore_index=True)
combined_FINAL_QP = pd.concat([FINAL_QP1, FINAL_QP2, FINAL_QP3, FINAL_QP4, FINAL_QP5, FINAL_QP6, FINAL_QP7, FINAL_QP8, FINAL_QP9, FINAL_QP10], ignore_index=True)

In [14]:
print(combined_train_df.shape)
print(combined_train_df_onlyGhost.shape)
print(combined_LABEL.shape)
print(combined_MAE.shape)
print(combined_FINAL_QP.shape)

(6000, 6)
(6000, 44)
(6000, 1)
(6000, 1)
(6000, 1)


In [15]:
print("Before scaling:")
print("Combined Train DF:")
print(combined_train_df.head())
print("Combined Train DF Only Ghost:")
print(combined_train_df_onlyGhost.head())

Before scaling:
Combined Train DF:
  FINAL_QP    KLD_PU  KLD_LUMA KLD_CHROMA    RATIO1    RATIO2
0       10  0.000457  0.000857   0.001671  0.117086  0.073309
1       16   0.00221  0.000518   0.001766  0.040061  0.022211
2       20  0.001292  0.000632   0.000502  0.027687  0.019464
3       24  0.000438  0.001074     0.0006  0.015096  0.008605
4       27    0.0001  0.000633   0.001434  0.238281  0.190728
Combined Train DF Only Ghost:
  FINAL_QP PU1_64 PU1_32 PU1_16  PU1_8  PU1_4 PU2_64 PU2_32 PU2_16  PU2_8  PU2_4 LU1_0  LU1_1 LU1_9 LU1_10 LU1_11 LU1_25 LU1_26 LU1_27 LU2_0  LU2_1 LU2_9 LU2_10 LU2_11 LU2_25 LU2_26 LU2_27  CH1_0  CH1_1 CH1_10 CH1_26 CH1_34 CH1_36  CH2_0  CH2_1 CH2_10 CH2_26 CH2_34 CH2_36    KLD_PU  KLD_LUMA KLD_CHROMA    RATIO1    RATIO2
0       10      0  14848   9760   7592  27800      0  14720   9744   7068  28468  6109  10992  2540   4598   1669   1943   2451   1757  6089  11990  2542   4709   1652   1998   2396   1774  14824  11408   8728   8124   1896  15020  15816  

In [16]:
def process_results_to_lists(train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP, scaler_main=None, scaler_ghost=None, fit_scaler=True):

    scaler_main = MinMaxScaler()
    scaler_ghost = MinMaxScaler()
    X_train = scaler_main.fit_transform(train_df)
    X_train_onlyGhost = scaler_ghost.fit_transform(train_df_onlyGhost)

    MAE_array = MAE.values
    FINAL_QP_array = FINAL_QP.values
    Y_train = LABEL['LABEL'].astype(int)

    return X_train, X_train_onlyGhost, MAE_array, FINAL_QP_array, Y_train, scaler_main, scaler_ghost

# スケーリングの処理
X_train, X_train_onlyGhost, MAE_array, FINAL_QP_array, Y_train, scaler_main, scaler_ghost = process_results_to_lists(
    combined_train_df, combined_train_df_onlyGhost, combined_LABEL, combined_MAE, combined_FINAL_QP, fit_scaler=True
)

# スケーリング後のデータを表示
print("After scaling:")
print("X_train:")
print(pd.DataFrame(X_train).head())
print("X_train_onlyGhost:")
print(pd.DataFrame(X_train_onlyGhost).head())

# データを元に戻すための関数
def restore_data_to_original_order(data, num_splits, original_lengths):
    restored_data = []
    start_index = 0
    for length in original_lengths:
        restored_data.append(data[start_index:start_index + length])
        start_index += length
    return restored_data

# 元のデータフレームの長さ
original_lengths = [len(train_df1), len(train_df2), len(train_df3), len(train_df4), len(train_df5), 
                    len(train_df6), len(train_df7), len(train_df8), len(train_df9), len(train_df10)]


# リストを初期化
X_train_list = []
X_train_onlyGhost_list = []
MAE_list = []
FINAL_QP_list = []
Y_train_list = []

# データを元の順序に戻す
X_train_list = restore_data_to_original_order(X_train, 10, original_lengths)
X_train_onlyGhost_list = restore_data_to_original_order(X_train_onlyGhost, 10, original_lengths)
MAE_list = restore_data_to_original_order(MAE_array, 10, original_lengths)
FINAL_QP_list = restore_data_to_original_order(FINAL_QP_array, 10, original_lengths)
Y_train_list = restore_data_to_original_order(Y_train, 10, original_lengths)

# 確認用の出力
for i in range(10):
    print(f"Length of X_train_restored[{i}]: {len(X_train_list[i])}")
    print(f"Length of X_train_onlyGhost_restored[{i}]: {len(X_train_onlyGhost_list[i])}")
    print(f"Length of MAE_restored[{i}]: {len(MAE_list[i])}")
    print(f"Length of FINAL_QP_restored[{i}]: {len(FINAL_QP_list[i])}")
    print(f"Length of Y_train_restored[{i}]: {len(Y_train_list[i])}")

After scaling:
X_train:
       0         1         2         3         4         5
0  0.125  0.012172  0.004898  0.017093  0.115655  0.072245
1  0.275  0.058844  0.002940  0.018062  0.038501  0.021082
2  0.375  0.034419  0.003596  0.005108  0.026107  0.018332
3  0.475  0.011666  0.006148  0.006112  0.013495  0.007460
4  0.550  0.002656  0.003601  0.014656  0.237053  0.189815
X_train_onlyGhost:
      0    1         2         3         4         5    6         7         8         9         10        11        12        13        14        15        16        17        18        19        20        21        22        23        24        25        26        27        28        29        30        31        32        33        34        35        36        37        38        39        40        41        42        43
0  0.125  0.0  0.251900  0.290200  0.263172  0.484422  0.0  0.249729  0.319015  0.241262  0.494751  0.094179  0.270190  0.194845  0.197464  0.138358  0.135523  0.085602  0.18

In [17]:
# Cの範囲を指定
C_values = {'C': [0.01, 0.1, 1, 10, 100, 1000, 2000, 3000, 4000, 5000]}
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# 結果のデータフレームを初期化
results = pd.DataFrame(columns=['C_RBF', 'Score_RBF', 'tnr_rbf', 'tpr_rbf',
                                'C_LINEAR', 'Score_LINEAR', 'tnr_linear', 'tpr_linear',
                                'C_OG_RBF', 'Score_OG_RBF', 'tnr_og_rbf', 'tpr_og_rbf',
                                'C_OG_LINEAR', 'Score_OG_LINEAR', 'tnr_og_linear', 'tpr_og_linear',
                                'Threshold', 'Score_old', 'tnr_old', 'tpr_old'])

X_index = np.arange(10)  # インデックスとして0から9までの数字を用意

# ループで各分割のtrain_idsとtest_idsを取得
for fold, (train_ids, test_ids) in enumerate(kfold.split(X_index)):
    print(f"<Fold-{fold+1}>")
    print("Train indices:", train_ids)
    print("Test indices:", test_ids)
        
    test_data = [X_train_list[i] for i in test_ids]
    test_data_OG = [X_train_onlyGhost_list[i] for i in test_ids]
    test_label = [Y_train_list[i] for i in test_ids]
    
    train_data = [X_train_list[i] for i in train_ids]
    train_data_OG = [X_train_onlyGhost_list[i] for i in train_ids]
    train_label = [Y_train_list[i] for i in train_ids]
    
    train_data = [item for data in train_data for item in data]
    train_data_OG = [item for data in train_data_OG for item in data]
    train_label = [item for data in train_label for item in data]
    
    X_train, X_val, Y_train, Y_val = train_test_split(train_data, train_label, test_size=600, random_state=42)
    X_train_OG, X_val_OG, _, _ = train_test_split(train_data_OG, train_label, test_size=600, random_state=42)
    
    test_data = [item for data in test_data for item in data]
    test_data_OG = [item for data in test_data_OG for item in data]
    test_label = [item for data in test_label for item in data]

    MAE_data = [MAE_list[i] for i in test_ids]
    MAE_data = [item for data in MAE_data for item in data]
    
    FINAL_QP_data = [FINAL_QP_list[i] for i in test_ids]
    FINAL_QP_data = [item for data in FINAL_QP_data for item in data]
    
    best_threshold = 0
    best_accuracy = 0
    best_predicted_labels = []
    best_ground_truth_labels = []
                
    for threshold in np.arange(0.00, 1.01, 0.01):
        results_old = np.array([is_double_compressed(MAE_data[i], FINAL_QP_data[i], threshold) for i in range(600)])
        predicted_labels = results_old.astype(int)
        ground_truth_labels = np.array(test_label)
        accuracy = np.sum(ground_truth_labels == predicted_labels) / len(ground_truth_labels)
    
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold
            best_predicted_labels = predicted_labels
            best_ground_truth_labels = ground_truth_labels
    
    best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = 0, None, None
    best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = 0, None, None
    
    best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = 0, None, None
    best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = 0, None, None
        
    for C_value in C_values['C']:    
        svm_model_RBF = SVC(kernel='rbf', C=C_value)
        svm_model_onlyGhost_RBF = SVC(kernel='rbf', C=C_value)
        
        svm_model_LINEAR = SVC(kernel='linear', C=C_value)
        svm_model_onlyGhost_LINEAR = SVC(kernel='linear', C=C_value)

        svm_model_RBF.fit(X_train, Y_train)
        svm_model_onlyGhost_RBF.fit(X_train_OG, Y_train)
        
        svm_model_LINEAR.fit(X_train, Y_train)
        svm_model_onlyGhost_LINEAR.fit(X_train_OG, Y_train)

        val_accuracy_RBF = accuracy_score(Y_val, svm_model_RBF.predict(X_val))
        val_accuracy_onlyGhost_RBF = accuracy_score(Y_val, svm_model_onlyGhost_RBF.predict(X_val_OG))
        
        val_accuracy_LINEAR = accuracy_score(Y_val, svm_model_LINEAR.predict(X_val))
        val_accuracy_onlyGhost_LINEAR = accuracy_score(Y_val, svm_model_onlyGhost_LINEAR.predict(X_val_OG))

        if val_accuracy_RBF > best_val_score_RBF:
            best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = val_accuracy_RBF, svm_model_RBF, C_value

        if val_accuracy_onlyGhost_RBF > best_val_score_onlyGhost_RBF:
            best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = val_accuracy_onlyGhost_RBF, svm_model_onlyGhost_RBF, C_value
            
        if val_accuracy_LINEAR > best_val_score_LINEAR:
            best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = val_accuracy_LINEAR, svm_model_LINEAR, C_value

        if val_accuracy_onlyGhost_LINEAR > best_val_score_onlyGhost_LINEAR:
            best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = val_accuracy_onlyGhost_LINEAR, svm_model_onlyGhost_LINEAR, C_value

    test_predictions_RBF = best_svm_model_RBF.predict(test_data)
    test_accuracy_RBF = accuracy_score(test_label, test_predictions_RBF)
    report_RBF = classification_report(test_label, test_predictions_RBF, digits=4)
    conf_matrix = confusion_matrix(test_label, test_predictions_RBF)
    tnr_rbf = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_rbf = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary_RBF:\n{report_RBF}')
    
    test_predictions_LINEAR = best_svm_model_LINEAR.predict(test_data)
    test_accuracy_LINEAR = accuracy_score(test_label, test_predictions_LINEAR)
    report_LINEAR = classification_report(test_label, test_predictions_LINEAR, digits=4)
    conf_matrix = confusion_matrix(test_label, test_predictions_LINEAR)
    tnr_linear = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_linear = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary_LINEAR:\n{report_LINEAR}')
        
    test_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(test_data_OG)
    test_accuracy_onlyGhost_RBF = accuracy_score(test_label, test_predictions_onlyGhost_RBF)
    report_onlyGhost_RBF = classification_report(test_label, test_predictions_onlyGhost_RBF, digits=4)
    conf_matrix = confusion_matrix(test_label, test_predictions_onlyGhost_RBF)
    tnr_og_rbf = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_og_rbf = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary2_RBF:\n{report_onlyGhost_RBF}')
    
    test_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(test_data_OG)
    test_accuracy_onlyGhost_LINEAR = accuracy_score(test_label, test_predictions_onlyGhost_LINEAR)
    report_onlyGhost_LINEAR = classification_report(test_label, test_predictions_onlyGhost_LINEAR, digits=4)
    conf_matrix = confusion_matrix(test_label, test_predictions_onlyGhost_LINEAR)
    tnr_og_linear = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_og_linear = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary2_LINEAR:\n{report_onlyGhost_LINEAR}')
    
    report_old = classification_report(best_ground_truth_labels, best_predicted_labels, labels=[0,1], target_names=['0', '1'], zero_division=0, digits=4)
    conf_matrix = confusion_matrix(best_ground_truth_labels, best_predicted_labels)
    tnr_old = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_old = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary old_model:\n{report_old}')
    
    result_row = {'C_RBF': best_c_value_RBF, 'Score_RBF': test_accuracy_RBF, 'tnr_rbf': tnr_rbf, 'tpr_rbf': tpr_rbf,
              'C_LINEAR': best_c_value_LINEAR, 'Score_LINEAR': test_accuracy_LINEAR, 'tnr_linear': tnr_linear, 'tpr_linear': tpr_linear,
              'C_OG_RBF': best_c_value_onlyGhost_RBF, 'Score_OG_RBF': test_accuracy_onlyGhost_RBF, 'tnr_og_rbf': tnr_og_rbf, 'tpr_og_rbf': tpr_og_rbf,
              'C_OG_LINEAR': best_c_value_onlyGhost_LINEAR, 'Score_OG_LINEAR': test_accuracy_onlyGhost_LINEAR, 'tnr_og_linear': tnr_og_linear, 'tpr_og_linear': tpr_og_linear,
              'Threshold': best_threshold, 'Score_old': best_accuracy, 'tnr_old': tnr_old, 'tpr_old': tpr_old}

    results = pd.concat([results, pd.DataFrame([result_row])], ignore_index=True)


<Fold-1>
Train indices: [0 1 2 3 4 5 6 7 9]
Test indices: [8]
Summary_RBF:
              precision    recall  f1-score   support

           0     0.7544    0.8500    0.7994       300
           1     0.8282    0.7233    0.7722       300

    accuracy                         0.7867       600
   macro avg     0.7913    0.7867    0.7858       600
weighted avg     0.7913    0.7867    0.7858       600

Summary_LINEAR:
              precision    recall  f1-score   support

           0     0.6303    0.8467    0.7226       300
           1     0.7665    0.5033    0.6076       300

    accuracy                         0.6750       600
   macro avg     0.6984    0.6750    0.6651       600
weighted avg     0.6984    0.6750    0.6651       600

Summary2_RBF:
              precision    recall  f1-score   support

           0     0.8133    0.8133    0.8133       300
           1     0.8133    0.8133    0.8133       300

    accuracy                         0.8133       600
   macro avg     0.8133

In [18]:
# 各統計情報を100倍して小数点第4位までの表記に変更
statistics_data = {
    'Model': ['RBF', 'LINEAR', 'RBF2', 'LINEAR2', 'OLD'],
    'Average TNR': [round(results['tnr_rbf'].mean() * 100, 2), round(results['tnr_linear'].mean() * 100, 2), round(results['tnr_og_rbf'].mean() * 100, 2), round(results['tnr_og_linear'].mean() * 100, 2), round(results['tnr_old'].mean() * 100, 2)],
    'Average TPR': [round(results['tpr_rbf'].mean() * 100, 2), round(results['tpr_linear'].mean() * 100, 2), round(results['tpr_og_rbf'].mean() * 100, 2), round(results['tpr_og_linear'].mean() * 100, 2), round(results['tpr_old'].mean() * 100, 2)],
    'Average Test Score': [round(results['Score_RBF'].mean() * 100, 2), round(results['Score_LINEAR'].mean() * 100, 2), round(results['Score_OG_RBF'].mean() * 100, 2), round(results['Score_OG_LINEAR'].mean() * 100, 2), round(results['Score_old'].mean() * 100, 2)],
    'Standard Deviation': [round(results['Score_RBF'].std() * 100, 2), round(results['Score_LINEAR'].std() * 100, 2), round(results['Score_OG_RBF'].std() * 100, 2), round(results['Score_OG_LINEAR'].std() * 100, 2), round(results['Score_old'].std() * 100, 2)],
    'Max Test Score': [round(results['Score_RBF'].max() * 100, 2), round(results['Score_LINEAR'].max() * 100, 2), round(results['Score_OG_RBF'].max() * 100, 2), round(results['Score_OG_LINEAR'].max() * 100, 2), round(results['Score_old'].max() * 100, 2)],
    'Min Test Score': [round(results['Score_RBF'].min() * 100, 2), round(results['Score_LINEAR'].min() * 100, 2), round(results['Score_OG_RBF'].min() * 100, 2), round(results['Score_OG_LINEAR'].min() * 100, 2), round(results['Score_old'].min() * 100, 2)],
}

# DataFrameを作成
statistics_df = pd.DataFrame(statistics_data)

# 表示
print(statistics_df)

     Model  Average TNR  Average TPR  Average Test Score  Standard Deviation  Max Test Score  Min Test Score
0      RBF        84.53        70.63               77.58                1.67           81.50           75.83
1   LINEAR        82.93        53.50               68.22                1.70           71.50           64.50
2     RBF2        84.13        79.57               81.85                1.41           83.83           79.67
3  LINEAR2        82.60        79.63               81.12                1.25           82.83           79.67
4      OLD        90.30        31.57               60.93                0.86           62.67           59.83


In [19]:
print(results['C_RBF'])
print(results['C_LINEAR'])

0    3000
1    3000
2    2000
3    3000
4    5000
5    2000
6    5000
7    1000
8    1000
9    5000
Name: C_RBF, dtype: object
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: C_LINEAR, dtype: object


In [20]:
statistics_df.to_csv('statistics_data9.csv', index=False)